# Initial Wholesale RE Analyses (melting the table for aggregation in tableau) April 18 2023

## when the dates are scatterd across in multiple columns and you are
## trying to have one column with the date that contains many rows of the 
## date as observations vs having them as different categories aka-columns

In [7]:
import numpy as np
import pandas as pd
import matplotlib as mtpl
import sklearn as sklearn
import seaborn as seaborn
import statsmodels as statsmodels
import csv

# ZHVI represents the typical home value for a region. Its calculated as a weighted average of the middle third of homes in a given region. zhvi how home values have changed over time. Gives us trends in the real estate market. It is a big index. 
# Zestimate uses variety of factors apart from just selling price : tax data, photographs, characteristics to estimate current and historical value. Zillow also looks at comparable. Looks at 35-65 percentile. Includes more homes since it gets all the zip codes. It is more complete.
### RegionID - unique identifier by zillow could be zip code, county or metro are
### Region Name - here it is the ZIP CODE for our work. It is usually the name or code of the geography being analyzed (zip code, county, metro area)
### County Name - Name of the county associated with the land analyzed
### Size Rank - rank of the geography's population size relative to other geographies in the dataset.

In [8]:
zip_data = pd.read_csv("zip_month.csv")

In [9]:
zip_data.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31
0,61148,0,8701,zip,NJ,NJ,Lakewood,"New York-Newark-Jersey City, NY-NJ-PA",Ocean County,132227.158462,...,513064.819267,521106.942820,526709.757886,529852.723408,532739.517232,536614.909083,539756.911224,542498.833639,544796.246875,548908.778223
1,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,222152.961968,...,483238.626925,487008.197655,487740.644864,486249.969249,484563.782495,483155.040379,481172.398003,478960.458716,477829.581123,478617.619647
2,91940,2,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,112898.162200,...,279587.901824,282543.997782,284092.001252,284287.573977,283684.397980,282690.504761,281117.092555,279305.188642,277752.055947,276950.323631
3,62080,3,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,72326.248058,...,323320.422839,323582.127606,322854.935372,321141.219564,319297.240310,318652.879789,317396.440188,317356.363695,317934.056006,320049.843886
4,92593,4,78660,zip,TX,TX,Pflugerville,"Austin-Round Rock-Georgetown, TX",Travis County,181767.202252,...,463719.895826,463689.682953,460142.037448,454155.731421,447851.951556,441110.142904,434006.030604,427856.292727,422973.617117,419470.057769


In [10]:
pa_data = zip_data[zip_data['State'] == 'PA']    

In [11]:
pa_data.shape

(1415, 288)

In [12]:
pa_data.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31
161,65787,165,19120,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,43656.208633,...,151659.800129,152772.730130,152783.204930,152295.225296,151892.114324,151654.583538,151028.223421,150968.433107,150323.716164,149553.611887
215,65791,219,19124,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,35446.241870,...,150896.948727,151150.151816,150344.763513,149605.992524,149441.004739,149761.901647,149460.276003,149748.602539,149457.148331,149461.654689
223,65133,227,17603,zip,PA,PA,Lancaster,"Lancaster, PA",Lancaster County,NaN,...,254209.868841,255509.559472,255635.340103,255358.802373,256045.302931,257553.803304,259237.481564,260912.865871,262353.050316,263844.885121
266,65779,270,19111,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,72185.343538,...,263220.221040,264122.678148,263113.277896,261371.500029,259874.275400,258740.700403,257250.380676,257110.764732,257272.946282,258089.374429
320,65810,325,19143,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,25540.826876,...,129952.671210,130255.605192,128885.475473,127250.468493,126776.584565,126738.424718,126453.225649,126945.127244,127200.825019,127525.840837


In [14]:
# look at all the unique values in the column for city
unique_pennsylvania = pa_data['City'].unique()

In [21]:
# count the values in state column
city_counts = pa_data['City'].value_counts()

In [25]:
import pprint
pprint.pprint(city_counts)

Philadelphia    46
Pittsburgh      35
Erie            10
Harrisburg       9
Reading          9
                ..
New Tripoli      1
Martinsburg      1
Centre Hall      1
Norwood          1
Oak Ridge        1
Name: City, Length: 1183, dtype: int64


In [26]:
 philly_data = pa_data[pa_data['City'] == 'Philadelphia']    

In [49]:
philly_data.shape

(46, 288)

In [32]:
phillyzips = philly_data['RegionName'].nunique()
print(phillyzips)

46


In [31]:
print(philly_data)

       RegionID  SizeRank  RegionName RegionType StateName State  \
161       65787       165       19120        zip        PA    PA   
215       65791       219       19124        zip        PA    PA   
266       65779       270       19111        zip        PA    PA   
320       65810       325       19143        zip        PA    PA   
413       65816       418       19149        zip        PA    PA   
475       65801       480       19134        zip        PA    PA   
792       65807       801       19140        zip        PA    PA   
918       65772       927       19104        zip        PA    PA   
925       65815       934       19148        zip        PA    PA   
1143      65811      1154       19144        zip        PA    PA   
1207      65812      1219       19145        zip        PA    PA   
1246      65806      1259       19139        zip        PA    PA   
1300      65798      1316       19131        zip        PA    PA   
1857      65813      1884       19146        zip

In [34]:
nan_philly = philly_data.isna().sum()
print(nan_philly)

RegionID      0
SizeRank      0
RegionName    0
RegionType    0
StateName     0
             ..
2022-11-30    0
2022-12-31    0
2023-01-31    0
2023-02-28    0
2023-03-31    0
Length: 288, dtype: int64


In [35]:
# check for types of data
philly_data.dtypes

RegionID        int64
SizeRank        int64
RegionName      int64
RegionType     object
StateName      object
               ...   
2022-11-30    float64
2022-12-31    float64
2023-01-31    float64
2023-02-28    float64
2023-03-31    float64
Length: 288, dtype: object

In [48]:
philly_data.head(40)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31
161,65787,165,19120,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,43656.208633,...,151659.800129,152772.730130,152783.204930,152295.225296,151892.114324,151654.583538,151028.223421,150968.433107,150323.716164,149553.611887
215,65791,219,19124,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,35446.241870,...,150896.948727,151150.151816,150344.763513,149605.992524,149441.004739,149761.901647,149460.276003,149748.602539,149457.148331,149461.654689
266,65779,270,19111,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,72185.343538,...,263220.221040,264122.678148,263113.277896,261371.500029,259874.275400,258740.700403,257250.380676,257110.764732,257272.946282,258089.374429
320,65810,325,19143,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,25540.826876,...,129952.671210,130255.605192,128885.475473,127250.468493,126776.584565,126738.424718,126453.225649,126945.127244,127200.825019,127525.840837
413,65816,418,19149,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,61974.729160,...,223624.928868,224153.366079,223342.625111,221864.890899,220436.978309,219452.235535,218619.243149,218334.335653,217740.576820,217376.632876
475,65801,480,19134,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,35997.807216,...,110515.808758,110589.133249,109226.617949,107570.363530,107006.845554,106744.081602,105917.791663,104846.750274,103414.691884,102406.719549
792,65807,801,19140,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,9285.546523,...,79408.939253,80460.286759,80160.204374,79375.706403,79426.397887,79510.493085,79247.946089,79058.575263,78840.557091,79067.803042
918,65772,927,19104,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,38985.952560,...,217150.944885,217819.666817,216468.480579,214753.758234,213970.876742,214283.708986,214049.048879,214436.650685,213292.900440,211648.243324
925,65815,934,19148,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,46765.903948,...,246964.758834,248528.963393,248724.531042,247857.268997,247224.091089,246946.413681,246558.895836,246680.100942,246160.063747,245388.835658
1143,65811,1154,19144,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,41783.991583,...,176142.573290,176214.007961,173928.474229,171272.273267,169959.415110,169479.047557,169177.633064,170125.545325,170860.934801,171377.708971


In [37]:
# load into excel to view everything for a quick check. 
philly_data.to_excel('philly_data.xlsx', index=False)

In [39]:
phillydata = pd.read_csv("fixedphilly_data.csv")

In [40]:
# get the median prices for each zip
median_prices = phillydata.median(axis=1)
print(median_prices)

0      75068.329535
1      69221.114355
2     142679.603200
3      41820.737160
4     123172.663850
5      54692.995330
6      21442.234215
7      70548.362845
8     140146.235600
9      70885.299815
10    106410.660600
11     34190.437065
12     63608.601585
13    163661.612400
14    292865.463000
15    216498.099200
16    126057.044200
17     99762.791480
18    216473.213150
19     34012.865430
20    184424.176450
21     58823.090205
22     16753.622330
23    179867.894200
24    213546.581250
25     91253.423765
26    177632.298300
27     66359.782065
28     42878.311510
29    190121.855100
30    294431.513350
31    138586.123100
32     16659.966635
33    421906.547550
34    129681.335650
35    102366.070650
36    111175.154800
37    291315.462300
38    221408.020500
39    109173.816050
40    311597.800300
41    177882.021200
42    431182.443700
43    104784.967150
44    210125.789300
45    334194.263050
dtype: float64


/var/folders/25/tj4ctqz551l05krbrgw2f6r40000gn/T/ipykernel_54906/3637677312.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  median_prices = phillydata.median(axis=1)


## you can melt/pivot the current table or just work with Tableau

### go ahead and melt the table as it will make tableau aggregations easier

In [64]:
phillydata.dtypes

RegionID        int64
SizeRank        int64
RegionName      int64
RegionType     object
StateName      object
               ...   
2022-11-30    float64
2022-12-31    float64
2023-01-31    float64
2023-02-28    float64
2023-03-31    float64
Length: 288, dtype: object

In [87]:
# id-vars are the column to leave unchanged
# var_name will have the new column with the columns we are trying to change to observations
# value_name will just have the content in the table as all values within one column

meltphillydata = pd.melt(frame = phillydata, id_vars=['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'State', 'City', 'Metro', 'CountyName'],
                        var_name="year-month", value_name="price")

In [88]:
meltphillydata.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,year-month,price
0,65787,165,19120,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,43656.20863
1,65791,219,19124,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,35446.24187
2,65779,270,19111,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,72185.34354
3,65810,325,19143,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,25540.82688
4,65816,418,19149,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,61974.72916


In [89]:
meltphillydata.shape

(12834, 11)

In [90]:
meltphillydata['year-month'].str.split('-').head()

0    [2000, 01, 31]
1    [2000, 01, 31]
2    [2000, 01, 31]
3    [2000, 01, 31]
4    [2000, 01, 31]
Name: year-month, dtype: object

In [91]:
meltphillydata['year-month'].str.split('-', expand = True)

,0,1,2
0,2000,01,31
1,2000,01,31
2,2000,01,31
3,2000,01,31
4,2000,01,31
...,...,...,...
12829,2023,03,31
12830,2023,03,31
12831,2023,03,31
12832,2023,03,31


In [92]:
## adding the year and month , date column into our dataset
meltphillydata[['year','month','day']] = meltphillydata['year-month'].str.split('-', expand=True)

In [93]:
meltphillydata.head(100)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,year-month,price,year,month,day
0,65787,165,19120,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,43656.208630,2000,01,31
1,65791,219,19124,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,35446.241870,2000,01,31
2,65779,270,19111,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,72185.343540,2000,01,31
3,65810,325,19143,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,25540.826880,2000,01,31
4,65816,418,19149,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,61974.729160,2000,01,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,65810,325,19143,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-03-31,25719.156930,2000,03,31
96,65816,418,19149,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-03-31,62001.678640,2000,03,31
97,65801,480,19134,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-03-31,36204.487100,2000,03,31
98,65807,801,19140,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-03-31,9334.155841,2000,03,31


In [94]:
meltphillydata.drop('day', axis = 1)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,year-month,price,year,month
0,65787,165,19120,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,43656.20863,2000,01
1,65791,219,19124,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,35446.24187,2000,01
2,65779,270,19111,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,72185.34354,2000,01
3,65810,325,19143,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,25540.82688,2000,01
4,65816,418,19149,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2000-01-31,61974.72916,2000,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12829,65796,9090,19129,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2023-03-31,288267.26870,2023,03
12830,65785,9199,19118,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2023-03-31,689747.58310,2023,03
12831,65804,10638,19137,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2023-03-31,197681.15830,2023,03
12832,65794,12744,19127,zip,PA,PA,Philadelphia,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia County,2023-03-31,301735.93190,2023,03


In [95]:
meltphillydata['year-month'] = pd.to_datetime(meltphillydata['year-month'])

In [96]:
meltphillydata['year'] = pd.to_datetime(meltphillydata['year'])

In [103]:
missingmonths = meltphillydata['month'].isna()

In [104]:
print(meltphillydata[missingmonths])

Empty DataFrame
Columns: [RegionID, SizeRank, RegionName, RegionType, StateName, State, City, Metro, CountyName, year-month, price, year, month, day]
Index: []


In [107]:
meltphillydata['month'] = meltphillydata['month'].astype('category')

In [108]:
print(meltphillydata.dtypes)

RegionID               int64
SizeRank               int64
RegionName             int64
RegionType            object
StateName             object
State                 object
City                  object
Metro                 object
CountyName            object
year-month    datetime64[ns]
price                float64
year          datetime64[ns]
month               category
day                   object
dtype: object


In [109]:
uniquemonths = meltphillydata['month'].unique()

In [110]:
print(uniquemonths)

['01', '02', '03', '04', '05', ..., '08', '09', '10', '11', '12']
Length: 12
Categories (12, object): ['01', '02', '03', '04', ..., '09', '10', '11', '12']


In [111]:
meltphillydata.to_excel('meltphilly_data.xlsx', index=False)

In [113]:
meltphillydata.groupby(['RegionName','year','month'])['price'].median()

RegionName  year        month
19102       2000-01-01  01       128544.2772
                        02       128877.0897
                        03       129619.7602
                        04       131114.5878
                        05       133436.1018
                                    ...     
19154       2023-01-01  08               NaN
                        09               NaN
                        10               NaN
                        11               NaN
                        12               NaN
Name: price, Length: 13248, dtype: float64

In [ ]:
# next try to observe any seasonal patterns in price dips.

In [114]:
meltphillydata.groupby(['RegionName','month'])['price'].median()

RegionName  month
19102       01       332983.36600
            02       332751.13270
            03       333757.70205
            04       331344.67040
            05       333222.45190
                         ...     
19154       08       185120.09750
            09       184448.41660
            10       184165.08760
            11       183197.37820
            12       184710.61880
Name: price, Length: 552, dtype: float64